## 2 - Export Data Assessment (Surface Marker)

##### Imports

In [1]:
import os
import pandas as pd
import numpy as np
import scipy as su
# from typing import Dict
# from scipy.signal import
# from scikit-learn import

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd

##### Data Loading

In [3]:
# Load ddfs into dictionary (exception for 'all_surf_pos', df)
root_dir = './aggregated_data'
def load_data(root_dir, exception=False):    # Get csv file reads into one dictionary
    data = {}
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            file_name = os.path.splitext(file)[0]   #file name without '.csv'
            if file.endswith('.csv') and file != 'all_surf_positions_HiDrive_Studie2.csv':
                data[file_name] = dd.read_csv(file_path)    #read and attach to dict
            elif exception and file == 'all_surf_positions_HiDrive_Studie2.csv':
                data [file_name] = pd.read_csv(file_path, converters=converters)    #read as normal and attach to dict
    return data
def parse(filedata): # Manually read the column
    output = []
    for line in filedata.split('\n'): # split into lines
        line = line.strip().rstrip(']').lstrip('[') #remove whitespace and brackets
        if not line:
            continue    #skip empty lines
        line = line.split() #split into cell
        row = []
        for cell in line:
            cell = cell.strip()     #remove whitespace
            if not cell.strip():
                continue    #skip empty cells
            row.append(float(cell)) #convert to float and add
        output.append(row)
    return output
converters = {
    "img_to_surf_trans": parse,
    "surf_to_img_trans": parse,
    "dist_img_to_surf_trans": parse,
    "surf_to_dist_img_trans": parse,
}

data = load_data(root_dir)

In [4]:
participant_ids = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
ddf = data['all_marker_detections'].iloc[:, 1:]
df = ddf.compute()

### 1. Inspection
Head & Tail,  Basic statistics,  Missing values

##### Head & tail

In [5]:
for pid in participant_ids:
    participant_df = df[df['participant_id'] == pid]
    display(f"Participant {pid}")
    display(participant_df.iloc[:, 1:].head())
    display(participant_df.iloc[:, 1:].tail())

'Participant 1'

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
0,0,apriltag_v3:tag36h11:1,971.042725,258.254608,1049.783203,259.982056,1048.784668,166.900436,969.649658,161.385681
1,0,apriltag_v3:tag36h11:2,209.350296,655.269226,272.727875,664.739136,259.347137,592.107422,195.022385,585.496094
2,0,apriltag_v3:tag36h11:3,932.797302,639.523560,1001.866150,627.168945,1014.362915,560.695190,942.566711,572.812073
3,1,apriltag_v3:tag36h11:1,971.630249,258.455353,1050.323975,260.219086,1049.285522,167.173950,970.399658,161.750046
4,1,apriltag_v3:tag36h11:2,209.480438,654.764771,272.868225,664.289124,259.821350,591.766968,195.277679,584.860229


,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
279310,116961,apriltag_v3:tag36h11:1,1137.503906,200.926498,1200.786133,213.872513,1201.607666,125.279564,1138.460083,107.637573
279311,116961,apriltag_v3:tag36h11:2,319.919250,563.245239,393.394958,573.792542,388.362091,494.033203,312.718262,486.560730
279312,116962,apriltag_v3:tag36h11:0,265.008392,183.293976,344.114441,173.725525,338.671082,77.942238,259.513214,92.602692
279313,116962,apriltag_v3:tag36h11:1,1137.971191,200.902542,1201.472412,213.842377,1201.791016,125.082176,1138.983521,107.642380
279314,116962,apriltag_v3:tag36h11:2,320.426910,563.360474,393.988525,573.985352,388.916107,494.292816,313.293488,486.541718


'Participant 2'

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
279315,0,apriltag_v3:tag36h11:1,893.462769,269.815826,955.759399,281.957977,965.616516,214.757584,903.369812,201.177521
279316,0,apriltag_v3:tag36h11:2,259.986694,482.185944,314.133148,494.324982,318.722595,432.545532,263.776947,421.970917
279317,0,apriltag_v3:tag36h11:3,832.436951,552.365295,889.271423,557.565125,904.127625,510.333771,844.829102,504.676178
279318,1,apriltag_v3:tag36h11:1,902.282837,260.631653,965.173584,271.613159,973.555298,203.913513,910.860657,191.460800
279319,1,apriltag_v3:tag36h11:2,271.597046,486.674500,326.273926,497.773743,329.604828,435.665009,274.227905,426.236023


...

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
209668,0,apriltag_v3:tag36h11:0,84.644836,295.563690,158.413666,285.618805,150.196915,168.562988,76.684456,187.960480
209669,0,apriltag_v3:tag36h11:1,1119.133789,268.047760,1191.770508,274.742493,1192.853271,168.085770,1121.904663,153.989258
209670,0,apriltag_v3:tag36h11:2,193.645432,707.460205,266.600433,721.244385,248.582458,642.347290,173.144501,629.354065
209671,0,apriltag_v3:tag36h11:3,1027.360962,691.705017,1096.915405,679.733032,1121.312500,609.992126,1049.413940,621.587769
209672,1,apriltag_v3:tag36h11:0,25.911097,338.706451,91.216827,330.206604,83.716629,214.415268,18.298174,231.720627


,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
48913,100469,apriltag_v3:tag36h11:2,710.791626,463.431519,782.802307,488.044556,800.779846,412.237335,725.193420,389.359222
48914,100470,apriltag_v3:tag36h11:0,722.850891,96.600906,810.638977,106.637413,810.598755,11.671686,721.210327,6.127400
48915,100470,apriltag_v3:tag36h11:2,705.074280,468.502747,776.912781,493.053864,794.928467,417.355438,719.526123,394.641632
48916,100471,apriltag_v3:tag36h11:0,716.003540,103.247856,803.578369,113.248070,804.160339,18.179089,715.192810,12.582260
48917,100471,apriltag_v3:tag36h11:2,697.547058,475.295807,769.076111,499.600281,787.177490,424.477081,711.978821,401.467651


'Participant 20'

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
48918,0,apriltag_v3:tag36h11:1,1018.894653,288.310272,1088.242554,287.919067,1091.940918,205.865387,1024.575684,195.899124
48919,0,apriltag_v3:tag36h11:2,275.301056,602.054016,340.768951,613.717102,337.845917,541.888977,270.975037,532.551697
48920,0,apriltag_v3:tag36h11:3,961.281555,632.561707,1025.089966,629.966980,1040.510254,565.127258,974.602051,567.250610
48921,1,apriltag_v3:tag36h11:0,241.495483,257.229523,311.393219,254.164261,311.053284,167.849304,240.903732,174.641907
48922,1,apriltag_v3:tag36h11:1,1020.903931,289.421021,1089.427856,287.183960,1093.012939,205.849808,1025.380371,195.614120


,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
359317,104720,apriltag_v3:tag36h11:2,424.011597,312.966492,492.358459,327.143341,504.460022,253.568939,434.673218,241.561310
359318,104720,apriltag_v3:tag36h11:3,1063.538452,445.726288,1116.334839,461.107849,1137.967529,399.452667,1084.459473,382.452332
359319,104721,apriltag_v3:tag36h11:1,1146.526001,127.470039,1199.325195,154.158203,1206.661011,87.858521,1154.953247,59.386562
359320,104721,apriltag_v3:tag36h11:2,418.960327,312.880981,487.658752,326.275909,498.872070,252.515976,429.081329,241.052704
359321,104721,apriltag_v3:tag36h11:3,1061.401001,440.721191,1114.740234,455.838013,1135.800293,393.621948,1081.950928,377.288147


##### Basic statistics
- 

In [6]:
for pid in participant_ids:
    participant_df = df[df['participant_id'] == pid]
    display(f"Participant {pid}")
    display(participant_df.iloc[:, 1:].describe())

'Participant 1'

,world_index,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
count,279315.000000,279315.000000,279315.000000,279315.000000,279315.000000,279315.000000,279315.000000,279315.000000,279315.000000
mean,53594.603727,616.398553,426.243525,688.134257,428.768162,687.228207,344.419710,614.578669,342.025042
std,33874.142841,394.594067,200.080088,393.171321,202.262675,400.666018,213.116598,402.352592,212.435915
min,0.000000,2.187053,24.360525,58.048580,46.234356,39.513950,-13.004531,-1.566993,-3.774724
25%,23959.500000,233.592293,249.184425,307.080139,249.444214,298.374039,150.411682,223.882950,150.532707
50%,50995.000000,437.632751,435.645020,520.250549,438.764130,525.213745,350.533203,439.494415,345.319794
75%,81848.000000,1004.200836,624.728943,1077.649414,629.762970,1083.800476,557.816437,1009.609985,552.942169
max,116962.000000,1229.624756,723.023315,1278.932129,732.559875,1281.454834,723.388428,1239.081177,721.905151


'Participant 2'

,world_index,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
count,233950.000000,233950.000000,233950.000000,233950.000000,233950.000000,233950.000000,233950.000000,233950.000000,233950.000000
mean,48898.905292,599.982591,493.152767,660.605038,497.219592,662.971190,432.633997,601.383173,428.721349
std,25745.944927,315.386431,150.982595,319.186777,152.516203,323.633174,161.838016,319.672680,161.426136
min,0.000000,2.330472,4.629794,22.551439,5.494884,0.895114,1.184230,-0.480664,2.040964
25%,27876.000000,270.165680,335.122986,327.576920,337.176826,325.294266,260.970360,267.018250,258.824875
50%,51413.000000,783.164276,566.583649,844.847748,570.572357,850.917755,514.388519,787.075867,510.890228
75%,70859.000000,884.159927,621.214752,946.374939,626.088425,953.926056,568.681366,890.428146,564.859741
max,90024.000000,1224.438477,722.253845,1264.856445,725.320251,1275.092529,707.447876,1242.637451,708.796448


'Participant 4'

...

,world_index,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
count,381724.000000,381724.000000,381724.000000,381724.000000,381724.000000,381724.000000,381724.000000,381724.000000,381724.000000
mean,54487.565175,584.505656,459.889458,655.658217,467.115050,664.740875,381.934580,592.923049,374.825589
std,30512.180755,417.951431,198.477710,415.433327,200.507747,422.017377,212.210548,425.186267,210.315655
min,0.000000,-0.440825,49.358315,21.755304,46.928368,4.371380,-5.265022,-0.212899,-0.284446
25%,27832.750000,175.530640,263.813148,247.765820,270.676414,248.848244,176.987896,174.563816,167.409386
50%,55717.500000,283.019073,577.032684,364.401505,596.075684,371.070480,513.466827,286.714752,496.943787
75%,80318.000000,996.952240,647.045456,1067.698334,658.491837,1083.733673,583.628677,1013.455444,573.129868
max,107250.000000,1226.306885,722.330444,1277.808472,721.414185,1281.472412,717.570679,1239.912842,719.395630


'Participant 19'

,world_index,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
count,197870.000000,197870.000000,197870.000000,197870.000000,197870.000000,197870.000000,197870.000000,197870.000000,197870.000000
mean,50488.921635,653.441755,423.360996,730.014747,425.123510,730.863390,330.263520,653.461388,328.621304
std,27895.623915,419.135717,194.851466,418.844715,197.066982,427.426381,211.203783,428.046257,209.907411
min,0.000000,-0.194068,13.801498,30.189011,20.451189,30.217623,-8.970974,-3.058805,-4.383344
25%,25593.000000,208.522625,254.619968,287.791580,254.431252,277.600311,144.383064,197.339695,143.621277
50%,53260.500000,875.327698,368.537003,961.972198,366.808426,967.023743,259.308456,877.670105,259.064240
75%,72979.000000,1031.532898,608.566895,1107.838989,613.877808,1119.260986,533.781937,1042.080933,529.667404
max,100471.000000,1224.317627,721.525574,1278.799072,738.090515,1282.644897,717.977783,1237.786255,709.622864


'Participant 20'

,world_index,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
count,310404.000000,310404.000000,310404.000000,310404.000000,310404.000000,310404.000000,310404.000000,310404.000000,310404.000000
mean,50399.544043,613.391156,403.657630,681.275888,409.554990,685.882722,331.334326,617.312187,324.946120
std,30578.360678,388.550740,183.504848,390.351498,183.901498,397.119960,190.552756,395.590780,191.616691
min,0.000000,3.162415,5.900505,14.853824,-696.932556,16.537979,-1.204548,0.426030,-3.097496
25%,23707.000000,201.269306,222.197262,267.624580,226.056358,262.738335,140.490967,196.166267,133.481476
50%,48972.000000,842.778809,466.127838,916.420074,474.656418,925.850098,398.217560,849.551117,390.434433
75%,77209.250000,969.775024,570.829483,1039.701477,577.773834,1052.405273,505.772141,982.584183,499.774811
max,104721.000000,1274.532837,722.669312,1274.032715,731.593140,1278.472412,712.279541,1257.703979,697.333191


##### Missing values
- No missing values

In [7]:
def missing_values(df):     # Compute missing values (count, pct and location)
    missing_data = df.isnull().sum()
    missing_percentage = ((missing_data / len(df)) * 100).apply(lambda x: f"{x:.2f}")
    missing_positions = df.apply(lambda x: ', '.join(str(idx) for idx in x.index[x.isnull()]), axis=0)

    missing_df = pd.DataFrame({     #create df to display
        'null_count': missing_data,
        'null_pct': missing_percentage,
        'location': missing_positions
    }).transpose()  # Transpose to have columns as indexes and new parameters as row indexes

    return missing_df

In [8]:
for pid in participant_ids:
    participant_df = df[df['participant_id'] == pid]
    display(f"Participant {pid}")
    display(missing_values(participant_df.iloc[:, 1:]))

'Participant 1'

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
null_count,0,0,0,0,0,0,0,0,0,0
null_pct,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
location,,,,,,,,,,


'Participant 2'

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
null_count,0,0,0,0,0,0,0,0,0,0
null_pct,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
location,,,,,,,,,,


'Participant 4'

...

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
null_count,0,0,0,0,0,0,0,0,0,0
null_pct,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
location,,,,,,,,,,


'Participant 19'

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
null_count,0,0,0,0,0,0,0,0,0,0
null_pct,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
location,,,,,,,,,,


'Participant 20'

,world_index,marker_uid,corner_0_x,corner_0_y,corner_1_x,corner_1_y,corner_2_x,corner_2_y,corner_3_x,corner_3_y
null_count,0,0,0,0,0,0,0,0,0,0
null_pct,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
location,,,,,,,,,,


### 2. (...)
(...)

##### (...)
- 